# 构造用于三维卷积神经网络的数据集
- 用周围19个像素的值取预测一个值，貌似太大了

In [1]:
import numpy as np
import tensorflow as tf
import scipy.io as sio
from sklearn.model_selection import train_test_split

In [2]:
data_path = os.path.join('E:\Eric_HSI\hyperspectral_datasets')
data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))
labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))

window_size=19
input_size=200

In [3]:
# 数据
# print(mat.keys())
features = data['data']
features_shape = features.shape
print(features.shape)
# 标签
labels = labels['groundT']
labels = np.reshape(labels, (-1, 1))   # 把标签变为2维, 总共21025

(145, 145, 200)


In [4]:
# 创建一个 (21025, 19, 19, 200) 的数组装数据
f2 = np.zeros((labels.shape[0], window_size, window_size, input_size), dtype='float32')

# parameter？？
parameter_b = np.array([i - window_size // 2 for i in range(window_size)])

In [5]:
parameter_b, features_shape, f2.shape

(array([-9, -8, -7, -6, -5, -4, -3, -2, -1,  0,  1,  2,  3,  4,  5,  6,  7,
         8,  9]),
 (145, 145, 200),
 (21025, 19, 19, 200))

In [6]:
# np.append 的正确用法 
num = []

# num = num.append(parameter_b[0] % features_shape[0])  # 这样是不对的
num.append(parameter_b[0] % features_shape[0])  # 这样才对
type(num)

list

In [7]:
# 求余运算
10 % 11 
-1 % 10         #  10 - 1 = 9
-2 % 10         #  1 - 2 = 8

a = 3
b = 5
c = (a, b)
type(c)

arr1 = np.arange(10)
arr2 = np.arange(10)

c = (arr1[1], arr2[2])
type(c)
c

(1, 2)

In [8]:
# 对下面的理解过程，到第九个像素的时候才对，刚开始的时候得到的patchs都是四周的像素的拼合，并不对
p = 0
for q in range(19):
    ind = (9 + parameter_b[1]) % features_shape[0], (9 + parameter_b[q]) % features_shape[0]
    print(ind)

(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(1, 12)
(1, 13)
(1, 14)
(1, 15)
(1, 16)
(1, 17)
(1, 18)


In [9]:
# 将原始像素以每个像素周围的8行8列组成patch，填入到初始创建的数组中，得到融入光谱信息patch
for i in range(features_shape[0]):
    for j in range(features_shape[1]):
        index = i * features_shape[1] + j
        for p in range(window_size):
            for q in range(window_size):
                f2[index][p][q] = \
                    features[(i + parameter_b[p]) % features_shape[0]][(j + parameter_b[q]) % features_shape[1]]

In [10]:
print(f2.shape)
f2[1][1][1].shape

(21025, 19, 19, 200)


(200,)

- 1、np.where(condition, x, y)，满足条件(condition)，输出x，不满足输出y。
- 2、np.where(condition)，只有条件 (condition)，没有x和y，则输出满足条件 (即非0) 元素的坐标。以tuple的形式给出。

- 3、np.delete(array,obj,axis)
    - array:需要处理的矩阵
    - obj:需要处理的位置，比如要删除的第一行或者第一行和第二行
    - axis:
    - 如果输入为None：array会先按行展开，然后按照obj，删除第obj-1(从0开始)位置的数，返回一个行矩阵。
    - 如果输入为0：按行删除
    - 如果输入为1：按列删除

In [11]:
# 删除所有没有标签的像素
print(labels.shape)
index = np.where(np.reshape(labels, (-1)) == 0)   # 返回坐标tuple(ndarray)
labels = np.delete(labels, index, axis=0)
print(labels.shape)

(21025, 1)
(10249, 1)


In [12]:
total = 0
for i in labels:
    if i == 0:
        total += 1
total

0

In [13]:
np.unique(labels) == 1  # 这是一种判断，集合where输出满足条件元素的坐标，也就是某个类别
np.where(np.array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]) == labels[0][0])

(array([3], dtype=int64),)

In [14]:
# labels[:20]

In [29]:
a = np.array([2,4,6,8,10])
np.where(a == 2)				# 返回索引
# (array([2, 3, 4]),)   
# (array([0], dtype=int64),)

(array([0], dtype=int64),)

In [36]:
# 验证标签 - 1
# 做一个实验验证下面的三行代码的作用
# 实验np，delete

# aa = np.delete(aa, [1,2], axis=0)
aa = np.array([1,2,3,4,5,6,7,8,9,10]).reshape(10,-1)    # 标签值
print(aa.shape)
temp = aa    # 类别数
print(temp)
for i in range(len(aa)):    
    aa[i][0] = np.where(temp == aa[i][0])[0][0]
print(aa.shape)
aa
# np.where(temp == aa[4][0])

(10, 1)
[[ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]]
(10, 1)


array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [30]:
tmp = np.unique(labels) 
tmp

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
      dtype=uint8)

In [31]:
tmp = np.unique(labels)   # array([1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16])
for i in range(len(labels)):
    labels[i][0] = np.where(tmp == labels[i][0])[0][0]  # (array([1], dtype=int64),) 第一个[0]索引出元祖中的ndarray，第二[0]索引出ndarray中的int值,处理后，所有的标签值减 1 ，因为本来中标签没用0，但是经过Len(labels)，np.where 定位索引位置，又从0开始，所以标签值-1

In [32]:
tmp = np.unique(labels) 
tmp

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
      dtype=uint8)

In [37]:
# labels[:20]
# labels.shape # (10249, 1)

In [38]:
f2 = np.delete(f2, index, axis=0)
labels = np.reshape(labels, (-1))  # 降维
labels.shape

(10249,)

In [39]:
dataset = [f2, labels]    # 拼接
(X, Y) = (dataset[0], dataset[1])  # -1,19,19,200
X = X.swapaxes(1, 3)      # 交换轴
print('X_shape: ', X.shape)
train_set = X[:, :, :, :, np.newaxis]     # 添加轴
print('TrainSet_shape: ', train_set.shape)


X_shape:  (10249, 200, 19, 19)
TrainSet_shape:  (10249, 200, 19, 19, 1)


In [40]:
print(Y.shape)

(10249,)


In [41]:
# 转化为类别变量
classes = 16
Y = tf.keras.utils.to_categorical(Y, classes)
print(Y.shape)

(10249, 16)


In [42]:
# 标准化
train_set = train_set.astype('float32')
train_set -= np.mean(train_set)
train_set /= np.max(train_set)

In [43]:
# Split the data
X_train_new, X_val_new, y_train_new, y_val_new = train_test_split(train_set, Y, test_size=0.8, random_state=4)

In [44]:
X_train_new.shape, X_val_new.shape, y_train_new.shape, y_val_new.shape

((2049, 200, 19, 19, 1), (8200, 200, 19, 19, 1), (2049, 16), (8200, 16))